In [19]:
import pandas as pd
import numpy as np

**TODO**
* Get historical data (reconsider the API provider)
* Figure out what and how to analyse
* Streamline the process, wrap it into async fastapi + docker

### What is being analysed:

**Descriptive statistics**
* Mean, median rating
* Std. deviations of the rating
* Mean rating by the day (with $95\%$ confidence interval)
* Box plot of ratings
* EDF for the rantings
* Raw points + EWMA over the whole observation time ($\alpha = 0.1$)
* (Spearman) correlation for ranking and weather conditions


**Inductive statistics**
* Inference of distribution parameters for the rankings (assume gamma distibution)
* General additive model, with all the parameters fitted with `scipy.curve_fit`:
    * AI infered functional form of each parameter
    * Fitted with scipy.curve_fit
    * Model of the form: $rating \sim \sum_{i=1}^{n} f_i(weather~factor_i)$
    * (With $95\%$ confidence interval)

In [72]:
raw_db = pd.read_csv("/Users/Misha/Documents/python_projects/day-rater-db/db.csv")

In [73]:
mask = np.random.choice([True,False],100,[90,10])
index = pd.date_range(start=raw_db.datetime[0],periods=100)[mask]
db = pd.DataFrame({},
                  columns=["day_rank","sunrise","sunset","season","atm_pressure","wind_speed","wind_direction","cloud_cover","rain_intensity","snow_intensity","sleet_intensity","magnetic_activity","temperature","feels_like","humidity"],
                  index=index
                 )

In [74]:
# shape = 3
# scale = 1
# mean = shape * scale = 7
arr = np.random.gamma(shape=7,scale=1,size=len(db)).round(1)
ranks = np.where(arr > 10, 10, arr)

db.day_rank = ranks

In [99]:
temperatures = np.random.normal(loc=25,scale=5,size=len(db)).round(1)
feels_like = (temperatures + np.random.normal(loc=0,scale=3,size=len(db))).round(1)

humidity = np.random.normal(loc=70,scale=10,size=len(db)).round(1)
humidity = np.where(humidity>100, 100, humidity)

db.temperature = temperatures
db.feels_like = feels_like
db.humidity = humidity
db.snow_intensity = 0
db.sleet_intensity = 0

In [100]:
db

,day_rank,sunrise,sunset,season,atm_pressure,wind_speed,wind_direction,cloud_cover,rain_intensity,snow_intensity,sleet_intensity,magnetic_activity,temperature,feels_like,humidity
2025-06-09 23:00:19,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,31.1,31.5,55.7
2025-06-10 23:00:19,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,22.0,26.8,71.1
2025-06-11 23:00:19,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,25.5,29.1,70.9
2025-06-17 23:00:19,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,22.6,20.9,62.4
2025-06-18 23:00:19,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,27.9,33.1,62.6
2025-06-19 23:00:19,7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,22.6,19.3,72.0
2025-06-20 23:00:19,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,24.7,25.6,73.8
2025-06-22 23:00:19,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,15.2,11.7,50.6
2025-06-23 23:00:19,5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,25.7,27.4,45.0
2025-06-24 23:00:19,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,26.0,24.4,75.3
